# Implementing Gradient Descent

In [2]:
import numpy as np

https://distill.pub/2017/momentum/

https://math.stackexchange.com/questions/78575/derivative-of-sigmoid-function-sigma-x-frac11e-x

## Sum of squared errors

Alternative loss function than the log-loss. Good option since:
- Error is always positive 
- Larger errors are penalized more than smaller errors.

The gradient of the SSE loss function can be derived through repetitive application of the chain rule. This ultimately leads to a product of the following items:
- Output error (error between actual label and prediction)
- The derivative of the activation function (a product of the linear combination of the weights and inputs)
- The inputs

The combination of the output error and the derivative of the activation function is considered the **error term**.

Below provides the code for the gradient descent step in the case of only one output unit:

In [3]:
# Defining the sigmoid function for activations
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Derivative of the sigmoid function
def sigmoid_prime(x):
    return sigmoid(x) * (1 - sigmoid(x))

# Input data
x = np.array([0.1, 0.3])
# Target
y = 0.2
# Input to output weights
weights = np.array([-0.8, 0.5])

# The learning rate, eta in the weight step equation
learnrate = 0.5

# the linear combination performed by the node (h in f(h) and f'(h))
h = x[0]*weights[0] + x[1]*weights[1]
# or h = np.dot(x, weights)

# The neural network output (y-hat)
nn_output = sigmoid(h)

# output error (y - y-hat)
error = y - nn_output

# output gradient (f'(h))
output_grad = sigmoid_prime(h)

# error term (lowercase delta)
error_term = error * output_grad

# Gradient descent step 
del_w = [ learnrate * error_term * x[0],
          learnrate * error_term * x[1]]
# or del_w = learnrate * error_term * x

## Implementing gradient descent

Considerations:
- Data must be standardized since the sigmoid function squashes really small and really large inputs. The gradient of really small and really large inputs (to the sigmoid function) is zero, which means that the gradient step will go to zero as well.
- Instead of using the SSE, use the MSE (i.e. the average SSE). This is because summing all the weight steps can lead to large updates that make the gradient descent diverge. To correct for this you would need a small learning rate. Instead by using the MSE the learning rate can stay stable.

The general algorithm:
- Set the weight **step** to zero
- For each record in the training data:
    - Make a forward pass through the network and calculate the output
    - Calculate the error term for the output unit
    - Update the weight **step**
- Update the weights by adding the learning rate times the updated weight step divided by the number of records.
- Repeat for e epochs

You can also update the weights on each record instead of averaging the weight steps after going through all the records.

Considerations for the numpy implementation:
- To initialize the weights it is important that they are:
    - Small such that the input to the sigmoid is in the linear region near 0 and squashed at the high/low ends
    - Random so that they all have different starting values and diverge, breaking symmetry.
    - Could use a Normal distribution of $ N(0, 1/\sqrt{n}) $, which keeps the input low for increasing numbers of input units.
- Numpy always defines arrays as row vectors. The transpose of a row vector will return a reow vector. Therefore, you need to use `arr[:,None]` to create a column vector.

In [ ]:
import numpy as np
from data_prep import features, targets, features_test, targets_test


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))

# TODO: We haven't provided the sigmoid_prime function like we did in
#       the previous lesson to encourage you to come up with a more
#       efficient solution. If you need a hint, check out the comments
#       in solution.py from the previous lecture.

# Use to same seed to make debugging easier
np.random.seed(42)

n_records, n_features = features.shape
last_loss = None

# Initialize weights
weights = np.random.normal(scale=1 / n_features**.5, size=n_features)

# Neural Network hyperparameters
epochs = 1000
learnrate = 0.5

for e in range(epochs):
    del_w = np.zeros(weights.shape)
    for x, y in zip(features.values, targets):
        # Loop through all records, x is the input, y is the target

        # Note: We haven't included the h variable from the previous
        #       lesson. You can add it if you want, or you can calculate
        #       the h together with the output

        # TODO: Calculate the output
        output = sigmoid(np.dot(x, weights))

        # TODO: Calculate the error
        error = y-output

        # TODO: Calculate the error term
        error_term = error * output * (1-output)

        # TODO: Calculate the change in weights for this sample
        #       and add it to the total weight change
        del_w += error_term * x

    # TODO: Update weights using the learning rate and the average change in weights
    weights += learnrate * del_w / n_records

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        out = sigmoid(np.dot(features, weights))
        loss = np.mean((out - targets) ** 2)
        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss


# Calculate accuracy on test data
tes_out = sigmoid(np.dot(features_test, weights))
predictions = tes_out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))

## Multi-layered perceptron

Notation:
- $w_{ij}$ denotes the weight on the edge from input unit $i$ and hidden unit $j$.
- The matrix of weights comprises:
    - For each row, the weights leading out of a single *input* unit (i.e. number of rows equal number of input units)
    - For each column, the weights leading into a single *hidden* unit (i.e. number of columns equals number of hidden units)

In [23]:
import numpy as np

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1/(1+np.exp(-x))

# Network size
N_input = 4
N_hidden = 3
N_output = 2

np.random.seed(42)
# Make some fake data
X = np.random.randn(4)

weights_input_to_hidden = np.random.normal(0, scale=0.1, size=(N_input, N_hidden))
weights_hidden_to_output = np.random.normal(0, scale=0.1, size=(N_hidden, N_output))


# TODO: Make a forward pass through the network

hidden_layer_in = np.dot(X, weights_input_to_hidden)
hidden_layer_out = sigmoid(hidden_layer_in)

print('Hidden-layer Output:')
print(hidden_layer_out)

output_layer_in = np.dot(hidden_layer_out, weights_hidden_to_output)
output_layer_out = sigmoid(output_layer_in)

print('Output-layer Output:')
print(output_layer_out)

Hidden-layer Output:
[0.41492192 0.42604313 0.5002434 ]
Output-layer Output:
[0.49815196 0.48539772]


## Backpropagation

- Previously have shown how to derive the gradient of the error function w.r.t. weights coming into the output unit. We saw that this is proportional to the *error term* which is the product of the error and the derivative of the sigmoid.
- For a multi-layered network, how do you get the error terms for the hidden layers? Using the chain rule, error for units is proportional to the error in the output layer times the weight between the units. This makes sense, since a unit in the hidden layer will contribute more to the error if the weight is higher. 
- Multiplying with the weights is the way you propagate inputs forward, and similarly how you propagate the errors backward. You can also view this by reverting the network and considering the errors as the inputs.

*Vanishing gradient problem:*
- The maximum derivative of the sigmoid function is 0.25, so the errors in the first layer are scaled down by 75% and in the next by 93.75%. So if you have a lot of layers, the weight steps will be reduced to tiny values towards the layers near the inputs. Therefore, those weights don't learn as much. Other activation functions treat this better.

In [41]:
import numpy as np


def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


x = np.array([0.5, 0.1, -0.2])
target = 0.6
learnrate = 0.5

weights_input_hidden = np.array([[0.5, -0.6],
                                 [0.1, -0.2],
                                 [0.1, 0.7]])

weights_hidden_output = np.array([0.1, -0.3])

## Forward pass
hidden_layer_input = np.dot(x, weights_input_hidden)
hidden_layer_output = sigmoid(hidden_layer_input)

output_layer_in = np.dot(hidden_layer_output, weights_hidden_output)
output = sigmoid(output_layer_in)

## Backwards pass
## TODO: Calculate output error
error = target - output

# TODO: Calculate error term for output layer
output_error_term = error * output * (1-output)

# TODO: Calculate error term for hidden layer
hidden_error_term = weights_hidden_output * output_error_term * hidden_layer_output * (1-hidden_layer_output)

# TODO: Calculate change in weights for hidden layer to output layer
delta_w_h_o = learnrate * output_error_term * hidden_layer_output

# TODO: Calculate change in weights for input layer to hidden layer
delta_w_i_h = learnrate * hidden_error_term * x[:,None]

print('Change in weights for hidden layer to output layer:')
print(delta_w_h_o)
print('Change in weights for input layer to hidden layer:')
print(delta_w_i_h)


Change in weights for hidden layer to output layer:
[0.00804047 0.00555918]
Change in weights for input layer to hidden layer:
[[ 1.77005547e-04 -5.11178506e-04]
 [ 3.54011093e-05 -1.02235701e-04]
 [-7.08022187e-05  2.04471402e-04]]


General algorithm for one hidden layer and one output unit:
- Set the weight steps for each layer to zero
    - The input to hidden weights
    - The hidden to output weights
- For each record in the training data:
    - Make a forward pass through the network, calculating the output
    - Calculate the error gradient in the output unit
    - Propagate the errors to the hidden layer
    - Update the weight steps
- Update the weights
- Repeat for $e$ epochs

In [ ]:
import numpy as np
from data_prep import features, targets, features_test, targets_test

np.random.seed(21)

def sigmoid(x):
    """
    Calculate sigmoid
    """
    return 1 / (1 + np.exp(-x))


# Hyperparameters
n_hidden = 2  # number of hidden units
epochs = 900
learnrate = 0.005

n_records, n_features = features.shape
last_loss = None
# Initialize weights
weights_input_hidden = np.random.normal(scale=1 / n_features ** .5,
                                        size=(n_features, n_hidden))
weights_hidden_output = np.random.normal(scale=1 / n_features ** .5,
                                         size=n_hidden)

for e in range(epochs):
    del_w_input_hidden = np.zeros(weights_input_hidden.shape)
    del_w_hidden_output = np.zeros(weights_hidden_output.shape)
    for x, y in zip(features.values, targets):
        ## Forward pass ##
        # TODO: Calculate the output
        hidden_input = np.dot(x, weights_input_hidden)
        hidden_output = sigmoid(hidden_input)
        output = sigmoid(np.dot(hidden_output, weights_hidden_output))

        ## Backward pass ##
        # TODO: Calculate the network's prediction error
        error = y - output

        # TODO: Calculate error term for the output unit
        output_error_term = error * output * (1-output)

        ## propagate errors to hidden layer

        # TODO: Calculate the hidden layer's contribution to the error
        hidden_error = output_error_term * weights_hidden_output
        
        # TODO: Calculate the error term for the hidden layer
        hidden_error_term = hidden_error * hidden_output * (1-hidden_output)
        
        # TODO: Update the change in weights
        del_w_hidden_output += output_error_term * hidden_output
        del_w_input_hidden += hidden_error_term * x[:,None]

    # TODO: Update weights  (don't forget to division by n_records or number of samples)
    weights_input_hidden += learnrate/ n_records * del_w_input_hidden
    weights_hidden_output += learnrate/ n_records * del_w_hidden_output

    # Printing out the mean square error on the training set
    if e % (epochs / 10) == 0:
        hidden_output = sigmoid(np.dot(x, weights_input_hidden))
        out = sigmoid(np.dot(hidden_output,
                             weights_hidden_output))
        loss = np.mean((out - targets) ** 2)

        if last_loss and last_loss < loss:
            print("Train loss: ", loss, "  WARNING - Loss Increasing")
        else:
            print("Train loss: ", loss)
        last_loss = loss

# Calculate accuracy on test data
hidden = sigmoid(np.dot(features_test, weights_input_hidden))
out = sigmoid(np.dot(hidden, weights_hidden_output))
predictions = out > 0.5
accuracy = np.mean(predictions == targets_test)
print("Prediction accuracy: {:.3f}".format(accuracy))
